In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Подключите необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 7)}); # you can change this if needed

Загрузите данные. Они находятся в директории input.

In [ ]:
df = pd.read_csv('../input/bigml_59c28831336c6604c800002a.csv', sep=',')

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe().T

*Как видно из информации о таблицу- данные очень хорошие, без выбросов.*

*Целевая переменная- churn - Отток. Характеризует перестал ли клиент пользоваться услугами компании. Принимает соответственно два значения: False- не верно, клиент всё ещё является клиентом. Лояльные абоненты
True- правда, клиент ушёл из компании.
Основная задача - выявить и оценить значимость характеристик клиентов, которые покидают компанию.*

In [ ]:
sns.boxplot(df['total night minutes'])

In [ ]:
 sns.boxplot(df['account length'])

In [ ]:
 sns.boxplot(df['total day charge'])

Все данные находятся достаточно близко к допустимым значениям, поэтому их можно не убирать и продолжать работу с датасетом в таком виде.

Каково распределение абонентов по целевому признаку **churn**? Назовите процент лояльных абонентов.

In [ ]:
# df['churn'].astype(int).hist(bins=2)

In [ ]:
df['churn'].value_counts(normalize=True)

*Процент лояльных пользователей ~ 85.5%*

Жители какого штата говорят по телефону больше всего? меньше всего? Обратите внимание, что нас интересуют все виды звонков.

In [ ]:
df.groupby('state')['total day minutes','total eve minutes','total night minutes','total intl minutes'].sum().sum(axis=1)

In [ ]:
df.groupby('state')['total day minutes','total eve minutes','total night minutes','total intl minutes'].sum().sum(axis=1).idxmax()

*Жители Западной Вирджинии разговаривают по телефону больше всего*

In [ ]:
df.groupby('state')['total day minutes','total eve minutes','total night minutes','total intl minutes'].sum().sum(axis=1).idxmin()

*Жители Калифорнии разговаривают по телефону меньше всего*

Верно ли, что люди, совершающие много международных звонков, приобретают **international plan**? Не забывайте о визуализации информации.

In [ ]:
plt.figure(figsize=(25,15))

sns.countplot(y='total intl calls', hue='international plan', data=df[(df['total intl calls']>df['total intl calls'].median())])

In [ ]:
plt.figure(figsize=(15, 8))

df[(df['total intl calls']>((df['total intl calls'].max()+df['total intl calls'].min())/2))].groupby('international plan')['total intl calls'].hist()

In [ ]:
# # df.groupby('total intl minutes')['international plan'].value_counts().plot()
# df[(df['international plan']=='yes')]['total intl calls'].value_counts().sort_index().plot()
# df[(df['international plan']=='no')]['total intl calls'].value_counts().sort_index().plot()
# plt.xlabel('total intl calls')
# plt.ylabel('count of people')
# plt.legend()

In [ ]:
# # df.groupby('total intl minutes')['international plan'].value_counts().plot()
# df[(df['international plan']=='yes')]['total intl minutes'].value_counts().sort_index().plot()
# df[(df['international plan']=='no')]['total intl minutes'].value_counts().sort_index().plot()
# plt.xlabel('total intl minutes')
# plt.ylabel('count of people')

In [ ]:
# plt.figure(figsize=(20, 50))

# sns.countplot(y='total intl minutes', hue='international plan', data=df);

In [ ]:
plt.figure(figsize=(25,15))

sns.countplot(y='total intl minutes', hue='international plan', data=df[(df['total intl minutes']>df['total intl minutes'].median())])

In [ ]:
plt.figure(figsize=(15, 8))

df[(df['total intl minutes']>((df['total intl minutes'].max()+df['total intl minutes'].min())/2))].groupby('international plan')['total intl minutes'].hist()

Как видим, исходя из графиков, не зависимо от количества международных звонков, ни от их общей продолжительности, количество абонентов, взявших международный план нигде не превысило количество абонентов с обычным планом. Хоть абоненты, звонящие по 15, 20 раз взяли междунардный план, никто из абонентов, совершающих 16-19 звонков не взял его. То, есть, люди много звонящие по международным линиям, в основном, редко берут международный план.

In [ ]:
df['international plan']=df['international plan'].map({'yes':1,'no':0})


In [ ]:
from scipy.stats import pointbiserialr
pointbiserialr(df['international plan'], df['total intl minutes'])

In [ ]:
pointbiserialr(df['international plan'], df['total intl calls'])

*Тесты на корреляцию подтверждают, что, на удивление, между количеством международных звонков и приобритением международного плана нет статистической взаимосвязи. Почти то же самое можно сказать относительно общей продолжительности международных звонков и наличием плана. Скорее всего, связь значима там из-за длинны выборки.*

Как связаны наличие голосовой почты и международного плана с лояльностью абонентов? Проверьте статистическую значимость этих связей.

In [ ]:
sns.countplot(y='international plan', hue='churn', data=df);

In [ ]:
pd.crosstab(df['international plan'], df['churn'])

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic
mosaic(df, ['international plan','churn'])

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['international plan'], df['churn']))

In [ ]:
fisher_exact(pd.crosstab(df['international plan'], df['churn']))

*Как видим, связь между наличием международного плана и оттока прямая и очень высокая. А крайне низкое значение p-value говорит о том, что связь статистически значима.*

In [ ]:
pd.crosstab(df['voice mail plan'], df['churn'])

In [ ]:
sns.countplot(y='voice mail plan', hue='churn', data=df);

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic
mosaic(df, ['voice mail plan','churn'])

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['voice mail plan'], df['churn']))

In [ ]:
fisher_exact(pd.crosstab(df['voice mail plan'], df['churn']))

*То есть, связь между наличием плана голосовой почты и оттоком также прямая, достаточно высокая и значимая, хоть и меньше, по сравнению с международным*

Что можно сказать о взаимосвязи количества обращений в колл-центр и лояльности? Не забывайте о визуализации информации.

In [ ]:
pointbiserialr(df['customer service calls'], df['churn'])

In [ ]:
plt.figure(figsize=(20, 15))
sns.countplot(y='customer service calls', hue='churn', data=df)

*Связь между оттоком и количеством звонков точно присутствует (очень низкое p-value), хоть и слабая. Связь прямая. То есть, чем больше звонил клиент в службу поддержки, тем больше вероятность, что он покинет компанию. Начиная, где-то с 5-ого звонка, вероятность, того, что он покинет компанию превышает вероятность того, что останется*

Какие из числовых признаков имеют наибольшую степень корреляции? Почему?

In [ ]:
numeric = ['account length', 'number vmail messages','total day minutes', 'total day calls','total day charge', 'total eve minutes', 'total eve calls','total eve charge','total night minutes','total night calls','total night charge','total intl minutes','total intl calls','total intl charge','customer service calls']
sns.pairplot(df[numeric])

In [ ]:
sns.heatmap(df[numeric].corr(method='spearman'));

In [ ]:
sns.heatmap(df[numeric].corr(method='pearson'));

In [ ]:
sns.heatmap(df[numeric].corr(method='kendall'));

Наибольшую степень корреляции, линеную, (кроме очевидной самого с собой) имеют признаки общего времени звонков и оплата клиента, что вполне логично. Также можно заметить намного более слабые, но тоже ожидаемые взаимосвязи между количеством звонков и общим временем, между количеством звонков и общей выручкой. Также заметна связь между количеством звонков в разное время (То есть, если клиент много звонил в одно время, есть вероятность, что он много звонил и в соседнее время (например, много звонков вечером- есть вероятность, что есть много звонков днём и ночью)). От сюда, соответственно, общее количество звонков слабо коррелирует с общим временем звонков и выручкой в другое время.

Связано ли время пользования аккаунтом с количеством обращений в колл-центр?

In [ ]:
sns.jointplot(x='customer service calls', y='account length', data=df);


In [ ]:
from scipy.stats import spearmanr, kendalltau
spearmanr(df['customer service calls'], df['account length'])

In [ ]:
kendalltau(df['customer service calls'], df['account length'])

Значение p-value очень высокое (намного больше 0.05), следовательно мы не можем утверждать о наличии взаимосвязи. 

In [ ]:
#Черновик, не стоит внимания. Коэфициент корреляции близок к 0, что указывает на отсутствие взаимосвязи, однако p-value, вероятность верности гипотезы H0 (то есть неверности нашей гипотезы HA: того, что взаимосвязь отсутствует),относительно близка к 1 (во всяком случае, ближе, чем к 0)=>есть вероятность, что есть связь. Однако, поскольку вероятность гипотезы H0 также статистически незначима (1-0.73), мы не можем утверждать и об этой зависимости.

Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

*Посмотрим, в каких штатах больше всего лояльных клиентов, а в каких больше всего отток.*

In [ ]:
df[df['churn']==True]['state'].value_counts()

In [ ]:
df[df['churn']==False]['state'].value_counts()

Больше всего отток в NJ (Нью-Джерси), меньше всего- в IA (Айова). Больше всего лояльных клиентов в WV (Западная Вирджиния), меньше всего- в CA (Калифорния).

Проверим, верно ли, что чем дольше клиент пользуется услугами, тем больше вероятность, что он не уйдёт из компании

In [ ]:
# sns.countplot(y='account length', hue='churn', data=df)
df.groupby('churn')['account length'].hist()
plt.ylabel('count of users')
plt.xlabel('account length')

In [ ]:
pointbiserialr(df['account length'], df['churn'])

Тест показывает, что зависимости между продолжительностью и пребывания клиентом и лояльности практически нет. Однако pvalue очень высокое, поэтому результат считать статисчиски значимым и говорить о зависимости исходя из него, не можем.

Проверим, верно ли, что чем больше клиент звонит, тем выше вероятность его лояльности.

In [ ]:
df['total minutes']=df[['total day minutes','total eve minutes','total night minutes','total intl minutes']].sum(axis=1)
df.groupby('churn')['account length'].hist()
plt.ylabel('count of users')
plt.xlabel('total minutes')

In [ ]:
pointbiserialr(df['total minutes'], df['churn'])

Как видим, зависимость между временем разговоров и оттоком есть, хотя и достаточно слабая.